In [22]:
import plotly.graph_objects as go
import pandas as pd
import json

# Generate synthetic data
total_cases = 1000
male_percentage = 0.6
female_percentage = 1 - male_percentage
male_cases = int(total_cases * male_percentage)
female_cases = total_cases - male_cases

data = pd.DataFrame({
    'category': ['Total', 'Male', 'Female'],
    'count': [total_cases, male_cases, female_cases],
    'sex': ['Total', 'Male', 'Female'] # Adding a 'sex' column for grouping
})

# Create the initial figure
fig = go.Figure(data=[go.Bar(
    x=data[data['category'] == 'Total']['category'],
    y=data[data['category'] == 'Total']['count'],
    text=[f'Total: {data[data["category"] == "Total"]["count"].iloc[0]}'],
    textposition='outside',
    marker_color='skyblue',
    name='Total',
    customdata=[['Total']], # Add customdata to identify the Total bar
    hovertemplate='Category: %{customdata[0]}<br>Count: %{y}<extra></extra>'
)])

# Define the dropdown menu (initially hidden or out of the way)
fig.update_layout(
    updatemenus=[
        {
            'buttons': list([
                dict(label='Total',
                     method='update',
                     args=[{'x': [data[data['category'] == 'Total']['category']],
                            'y': [data[data['category'] == 'Total']['count']],
                            'text': [f'Total: {data[data["category"] == "Total"]["count"].iloc[0]}']
                           },
                           {'title': 'Total HIV Cases',
                            'yaxis': {'title': 'Number of Cases'}}]),
                dict(label='By Sex',
                     method='update',
                     args=[{'x': [data[data['sex'] != 'Total']['sex']],
                            'y': [data[data['sex'] != 'Total']['count']],
                            'text': data[data['sex'] != 'Total'].apply(lambda row: f"{row['sex']}: {row['count']}", axis=1),
                            'marker': {'color': ['lightblue', 'lightcoral']},
                            'name': ['Male', 'Female']
                           },
                           {'title': 'HIV Cases by Sex',
                            'yaxis': {'title': 'Number of Cases'}}]),
            ]),
            'direction': 'down',
            'pad': {'r': 10, 't': 10},
            'showactive': False, # Initially don't show active button
            'x': -1, # Position the dropdown off-screen
            'xanchor': 'left',
            'y': 1.1,
            'yanchor': 'top'
        },
    ],
    title='Total HIV Cases',
    xaxis_title='',
    yaxis_title='Number of Cases'
)

# JavaScript function to handle clicks and trigger the 'By Sex' update
click_handler = """
<script type="text/javascript">
    var plotly_div = document.getElementById('plotly-div');
    plotly_div.on('plotly_click', function(data) {
        console.log('plotly_click event triggered:', data); // Log the entire data object
        if (data && data.points && data.points.length > 0) {
            var clickedCategory = data.points[0].x;
            var clickedCustomData = data.points[0].customdata;

            console.log('Clicked Category:', clickedCategory);
            console.log('Clicked Custom Data:', clickedCustomData);

            if (clickedCategory === 'Total' && clickedCustomData && clickedCustomData[0] === 'Total') {
                console.log('Condition met: Total bar clicked');
                Plotly.relayout('plotly-div', {
                    'data[0].x': [['Male', 'Female']],
                    'data[0].y': [{{male_cases}}, {{female_cases}}],
                    'data[0].text': [['Male: {{male_cases}}', 'Female: {{female_cases}}']],
                    'data[0].marker.color': [['lightblue', 'lightcoral']],
                    'data[0].name': ['Male', 'Female'],
                    'layout.title': 'HIV Cases by Sex',
                    'layout.yaxis.title': 'Number of Cases',
                    'config': {'transition': {'duration': 500, 'easing': 'cubic-in-out'}} // Add transition
                }).then(function() {
                    console.log('Plotly.relayout completed'); // Log when relayout finishes
                });
            } else {
                console.log('Condition not met');
            }
        } else {
            console.log('No click data available');
        }
    });
</script>
"""

# Embed the Plotly figure and the JavaScript click handler in HTML
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Interactive HIV Cases</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <div id="plotly-div"></div>
    {fig.to_html(full_html=False, include_plotlyjs='cdn')}
    {click_handler}
    <noscript>Your browser does not support JavaScript!</noscript>
</body>
</html>
"""

# Replace placeholders in JavaScript
html_content = html_content.replace('{{male_cases}}', str(male_cases))
html_content = html_content.replace('{{female_cases}}', str(female_cases))

# Save the HTML to a file
with open("interactive_hiv_cases_transition.html", "w") as f:
    f.write(html_content)

print("Interactive visualization with click transition saved to interactive_hiv_cases_transition.html")

Interactive visualization with click transition saved to interactive_hiv_cases_transition.html


In [27]:
import plotly.graph_objects as go
import pandas as pd

# Generate synthetic data
total_cases = 1000
male_percentage = 0.6
female_percentage = 1 - male_percentage
male_cases = int(total_cases * male_percentage)
female_cases = total_cases - male_cases

data = pd.DataFrame({
    'category': ['Total', 'Male', 'Female'],
    'count': [total_cases, male_cases, female_cases],
    'sex': ['Total', 'Male', 'Female']
})

# Create figure
fig = go.Figure()

# Add first (Total) trace
fig.add_trace(go.Bar(
    x=data[data['category'] == 'Total']['category'],
    y=data[data['category'] == 'Total']['count'],
    text=[f'Total: {data[data["category"] == "Total"]["count"].iloc[0]}'],
    textposition='outside',
    marker_color='skyblue',
    name='Total'
))

# Add second (By Sex) trace, initially invisible
fig.add_trace(go.Bar(
    x=data[data['sex'] != 'Total']['sex'],
    y=data[data['sex'] != 'Total']['count'],
    text=data[data['sex'] != 'Total'].apply(lambda row: f"{row['sex']}: {row['count']}", axis=1),
    textposition='outside',
    marker_color=['lightblue', 'lightcoral'],
    visible=False,
    name='By Sex'
))

# Layout
fig.update_layout(
    title='Total HIV Cases',
    xaxis_title='',
    yaxis_title='Number of Cases',
    updatemenus=[
        {
            'buttons': [
                {
                    'label': 'Total',
                    'method': 'update',
                    'args': [{'visible': [True, False]},
                             {'title': 'Total HIV Cases'}]
                },
                {
                    'label': 'By Sex',
                    'method': 'update',
                    'args': [{'visible': [False, True]},
                             {'title': 'HIV Cases by Sex'}]
                }
            ],
            'direction': 'down',
            'pad': {'r': 10, 't': 10},
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.1,
            'yanchor': 'top'
        }
    ]
)

# Save the figure first
fig.write_html("interactive_hiv_cases_dropdown.html", include_plotlyjs='cdn')

print("Interactive visualization saved. Now injecting click transition...")

# Inject custom JavaScript to handle clicks
with open("interactive_hiv_cases_dropdown.html", "r") as f:
    html = f.read()

# New custom script
custom_script = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var currentView = 0; // 0 = Total, 1 = By Sex

    var plot = document.getElementsByClassName('plotly-graph-div')[0];
    if (plot) {
        plot.on('plotly_click', function() {
            currentView = (currentView + 1) % 2;
            var update = {};
            if (currentView === 0) {
                update = {visible: [true, false]};
                Plotly.restyle(plot, update);
                Plotly.relayout(plot, {title: 'Total HIV Cases'});
            } else {
                update = {visible: [false, true]};
                Plotly.restyle(plot, update);
                Plotly.relayout(plot, {title: 'HIV Cases by Sex'});
            }
        });
    }
});
</script>
"""

# Add the custom script right before </body>
html = html.replace("</body>", custom_script + "\n</body>")

# Save modified HTML
with open("interactive_hiv_cases_dropdown.html", "w") as f:
    f.write(html)

print("Click-to-transition behavior injected successfully!")


Interactive visualization saved. Now injecting click transition...
Click-to-transition behavior injected successfully!


In [34]:
import plotly.graph_objects as go
import pandas as pd

# Assuming df_counts is already loaded

# Step 1: Clean column names
df_counts.columns = df_counts.columns.str.replace('\n', ' ').str.replace(' +', ' ', regex=True).str.strip()

# Step 2: Subset to the first row
first_row = df_counts.iloc[0]

# Step 3: Extract values
total_cases = int(first_row['New Diagnoses National Cases'])
male_cases = int(first_row['New Diagnoses Male Cases'])
female_cases = int(first_row['New Diagnoses Female Cases'])

# Step 4: Build new dataframe
data = pd.DataFrame({
    'category': ['Total', 'Male', 'Female'],
    'count': [total_cases, male_cases, female_cases],
    'sex': ['Total', 'Male', 'Female']
})

# Step 5: Build the figure
fig = go.Figure()

# Add first (Total) trace
fig.add_trace(go.Bar(
    x=data[data['category'] == 'Total']['category'],
    y=data[data['category'] == 'Total']['count'],
    text=[f'Total: {data[data["category"] == "Total"]["count"].iloc[0]}'],
    textposition='outside',
    marker_color='skyblue',
    name='Total'
))

# Add second (By Sex) trace, initially invisible
fig.add_trace(go.Bar(
    x=data[data['sex'] != 'Total']['sex'],
    y=data[data['sex'] != 'Total']['count'],
    text=data[data['sex'] != 'Total'].apply(lambda row: f"{row['sex']}: {row['count']}", axis=1),
    textposition='outside',
    marker_color=['lightblue', 'lightcoral'],
    visible=False,
    name='By Sex'
))

# Layout
fig.update_layout(
    title='Total HIV Cases',
    xaxis_title='',
    yaxis_title='Number of Cases',
    updatemenus=[
        {
            'buttons': [
                {
                    'label': 'Total',
                    'method': 'update',
                    'args': [{'visible': [True, False]},
                             {'title': 'Total HIV Cases'}]
                },
                {
                    'label': 'By Sex',
                    'method': 'update',
                    'args': [{'visible': [False, True]},
                             {'title': 'HIV Cases by Sex'}]
                }
            ],
            'direction': 'down',
            'pad': {'r': 10, 't': 10},
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.1,
            'yanchor': 'top'
        }
    ]
)

# Save the figure first
fig.write_html("interactive_hiv_cases_dropdown.html", include_plotlyjs='cdn')

print("Interactive visualization saved. Now injecting click transition...")

# Inject custom JavaScript to handle clicks
with open("interactive_hiv_cases_dropdown.html", "r") as f:
    html = f.read()

# New custom script
custom_script = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var currentView = 0;

    var plot = document.getElementsByClassName('plotly-graph-div')[0];
    if (plot) {
        plot.on('plotly_click', function() {
            currentView = (currentView + 1) % 2;
            var update = {};
            if (currentView === 0) {
                update = {visible: [true, false]};
                Plotly.restyle(plot, update);
                Plotly.relayout(plot, {title: 'Total HIV Cases'});
            } else {
                update = {visible: [false, true]};
                Plotly.restyle(plot, update);
                Plotly.relayout(plot, {title: 'HIV Cases by Sex'});
            }
        });
    }
});
</script>
"""

# Add the custom script right before </body>
html = html.replace("</body>", custom_script + "\n</body>")

# Save modified HTML
with open("interactive_hiv_cases_dropdown.html", "w") as f:
    f.write(html)

print("Click-to-transition behavior injected successfully!")


Interactive visualization saved. Now injecting click transition...
Click-to-transition behavior injected successfully!


In [38]:
df_counts = pd.read_csv("../../data/clean_data/transmission_counts.csv")
df_percentages = pd.read_csv("../../data/clean_data/transmission_percentages.csv")

In [39]:
df_counts.columns

Index(['Unnamed: 0', 'New Diagnoses\nNational Cases',
       'New Diagnoses\nMale Cases', 'New Diagnoses\nFemale Cases',
       'New Diagnoses\nMSM Cases', 'New Diagnoses\nMSM/IDU Cases',
       'New Diagnoses IDU\nCases',
       'New Diagnoses\nHeterosexual\nContact Cases',
       'New Diagnoses\nOther Transmission\nCategory Cases',
       'New Diagnoses\nMale and MSM\nCases',
       'New Diagnoses\nMale and IDU\nCases',
       'New Diagnoses\nMale and MSM/IDU\nCases',
       'New Diagnoses\nMale and Other\nTransmission\nCategory Cases',
       'New Diagnoses\nMale and\nHeterosexual\nContact Cases',
       'New Diagnoses\nFemale and\nHeterosexual\nContact Cases',
       'New Diagnoses\nFemale and IDU\nCases',
       'New Diagnoses\nFemale and Other\nTransmission\nCategory Cases',
       'Year'],
      dtype='object')

## Attempt 2: breaks into sex and then breakdown % by sex

In [37]:
import plotly.graph_objects as go
import pandas as pd

# Assuming df_counts and df_percentages are already loaded

# Step 1: Clean column names for both dataframes
df_counts.columns = df_counts.columns.str.replace('\n', ' ').str.replace(' +', ' ', regex=True).str.strip()

# Do the same for percentages dataframe
df_percentages = df_percentages.copy()  # Avoid modifying original
df_percentages.columns = df_percentages.columns.str.replace('\n', ' ').str.replace(' +', ' ', regex=True).str.strip()

# Step 2: Subset to the first row for both dataframes
first_row_counts = df_counts.iloc[0]
first_row_percentages = df_percentages.iloc[0]

# Step 3: Extract values for counts
total_cases = int(first_row_counts['New Diagnoses National Cases'])
male_cases = int(first_row_counts['New Diagnoses Male Cases'])
female_cases = int(first_row_counts['New Diagnoses Female Cases'])

# Extract percentage values and calculate counts for male transmission modes
male_msm_percent = float(first_row_percentages['New Diagnoses Male and MSM Percent'])
male_idu_percent = float(first_row_percentages['New Diagnoses Male and IDU Percent'])
male_msm_idu_percent = float(first_row_percentages['New Diagnoses Male and MSM/IDU Percent'])
male_other_percent = float(first_row_percentages['New Diagnoses Male and Other Transmission Category Percent'])
male_hetero_percent = float(first_row_percentages['New Diagnoses Male and Heterosexual Contact Percent'])

# Calculate actual counts for male transmission categories
male_msm_count = int(male_cases * male_msm_percent / 100)
male_idu_count = int(male_cases * male_idu_percent / 100)
male_msm_idu_count = int(male_cases * male_msm_idu_percent / 100)
male_other_count = int(male_cases * male_other_percent / 100)
male_hetero_count = int(male_cases * male_hetero_percent / 100)

# Extract percentage values and calculate counts for female transmission modes
female_hetero_percent = float(first_row_percentages['New Diagnoses Female and Heterosexual Contact Percent'])
female_idu_percent = float(first_row_percentages['New Diagnoses Female and IDU Percent'])
female_other_percent = float(first_row_percentages['New Diagnoses Female and Other Transmission Category Percent'])

# Calculate actual counts for female transmission categories
female_hetero_count = int(female_cases * female_hetero_percent / 100)
female_idu_count = int(female_cases * female_idu_percent / 100)
female_other_count = int(female_cases * female_other_percent / 100)

# Step 4: Build dataframes for all levels
# Level 1: Total
data_total = pd.DataFrame({
    'category': ['Total'],
    'count': [total_cases],
    'level': ['total']
})

# Level 2: By Sex
data_sex = pd.DataFrame({
    'category': ['Male', 'Female'],
    'count': [male_cases, female_cases],
    'level': ['sex', 'sex']
})

# Level 3a: Male Transmission Categories
data_male_trans = pd.DataFrame({
    'category': ['MSM', 'IDU', 'MSM/IDU', 'Other', 'Heterosexual'],
    'count': [male_msm_count, male_idu_count, male_msm_idu_count, male_other_count, male_hetero_count],
    'percent': [male_msm_percent, male_idu_percent, male_msm_idu_percent, male_other_percent, male_hetero_percent],
    'level': ['male_trans', 'male_trans', 'male_trans', 'male_trans', 'male_trans']
})

# Level 3b: Female Transmission Categories
data_female_trans = pd.DataFrame({
    'category': ['Heterosexual', 'IDU', 'Other'],
    'count': [female_hetero_count, female_idu_count, female_other_count],
    'percent': [female_hetero_percent, female_idu_percent, female_other_percent],
    'level': ['female_trans', 'female_trans', 'female_trans']
})

# Step 5: Build the figure with three traces (levels)
fig = go.Figure()

# Add first trace (Total)
fig.add_trace(go.Bar(
    x=data_total['category'],
    y=data_total['count'],
    text=[f'Total: {total_cases:,}'],
    textposition='outside',
    marker_color='skyblue',
    name='Total',
    hovertemplate='<b>%{x}</b><br>Count: %{y:,}<extra></extra>'
))

# Add second trace (By Sex)
fig.add_trace(go.Bar(
    x=data_sex['category'],
    y=data_sex['count'],
    text=[f'{sex}: {count:,}' for sex, count in zip(data_sex['category'], data_sex['count'])],
    textposition='outside',
    marker_color=['lightblue', 'lightcoral'],
    visible=False,
    name='By Sex',
    hovertemplate='<b>%{x}</b><br>Count: %{y:,}<extra></extra>'
))

# Add third trace (Male Transmission)
fig.add_trace(go.Bar(
    x=data_male_trans['category'],
    y=data_male_trans['count'],
    text=[f'{cat}: {count:,} ({percent}%)' for cat, count, percent in 
          zip(data_male_trans['category'], data_male_trans['count'], data_male_trans['percent'])],
    textposition='outside',
    marker_color=['rgb(214, 39, 40)', 'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 
                 'rgb(227, 119, 194)', 'rgb(127, 127, 127)'],
    visible=False,
    name='Male Transmission',
    hovertemplate='<b>%{x}</b><br>Count: %{y:,}<br>Percent: %{text}<extra></extra>'
))

# Add fourth trace (Female Transmission)
fig.add_trace(go.Bar(
    x=data_female_trans['category'],
    y=data_female_trans['count'],
    text=[f'{cat}: {count:,} ({percent}%)' for cat, count, percent in 
          zip(data_female_trans['category'], data_female_trans['count'], data_female_trans['percent'])],
    textposition='outside',
    marker_color=['rgb(127, 127, 127)', 'rgb(148, 103, 189)', 'rgb(227, 119, 194)'],
    visible=False,
    name='Female Transmission',
    hovertemplate='<b>%{x}</b><br>Count: %{y:,}<br>Percent: %{text}<extra></extra>'
))

# Layout
fig.update_layout(
    title='HIV Cases Breakdown',
    xaxis_title='',
    yaxis_title='Number of Cases',
    updatemenus=[
        {
            'buttons': [
                {
                    'label': 'Total',
                    'method': 'update',
                    'args': [{'visible': [True, False, False, False]},
                             {'title': 'Total HIV Cases'}]
                },
                {
                    'label': 'By Sex',
                    'method': 'update',
                    'args': [{'visible': [False, True, False, False]},
                             {'title': 'HIV Cases by Sex'}]
                },
                {
                    'label': 'Male Transmission',
                    'method': 'update',
                    'args': [{'visible': [False, False, True, False]},
                             {'title': 'HIV Cases by Male Transmission Category'}]
                },
                {
                    'label': 'Female Transmission',
                    'method': 'update',
                    'args': [{'visible': [False, False, False, True]},
                             {'title': 'HIV Cases by Female Transmission Category'}]
                }
            ],
            'direction': 'down',
            'pad': {'r': 10, 't': 10},
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.1,
            'yanchor': 'top'
        }
    ]
)

# Save the figure first
fig.write_html("interactive_hiv_cases_drill_down.html", include_plotlyjs='cdn')

print("Interactive visualization saved. Now injecting click transition...")

# Inject custom JavaScript to handle clicks
with open("interactive_hiv_cases_drill_down.html", "r") as f:
    html = f.read()

# New custom script with three levels of drill-down
custom_script = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var currentView = 0;
    var currentGender = null;
    var plot = document.getElementsByClassName('plotly-graph-div')[0];
    
    if (plot) {
        plot.on('plotly_click', function(data) {
            // Get the clicked point
            const clickedPoint = data.points[0];
            const clickedCategory = clickedPoint.x;
            
            if (currentView === 0) {
                // From Total to Sex breakdown
                currentView = 1;
                Plotly.restyle(plot, {visible: [false, true, false, false]});
                Plotly.relayout(plot, {title: 'HIV Cases by Sex'});
            } 
            else if (currentView === 1) {
                // From Sex to Transmission Category
                currentView = 2;
                if (clickedCategory === 'Male') {
                    currentGender = 'male';
                    Plotly.restyle(plot, {visible: [false, false, true, false]});
                    Plotly.relayout(plot, {title: 'HIV Cases by Male Transmission Category'});
                } else if (clickedCategory === 'Female') {
                    currentGender = 'female';
                    Plotly.restyle(plot, {visible: [false, false, false, true]});
                    Plotly.relayout(plot, {title: 'HIV Cases by Female Transmission Category'});
                }
            }
            else if (currentView === 2) {
                // From Transmission back to Sex breakdown
                currentView = 1;
                Plotly.restyle(plot, {visible: [false, true, false, false]});
                Plotly.relayout(plot, {title: 'HIV Cases by Sex'});
            }
        });
        
        // Add double-click to reset to total view
        plot.on('plotly_doubleclick', function() {
            currentView = 0;
            currentGender = null;
            Plotly.restyle(plot, {visible: [true, false, false, false]});
            Plotly.relayout(plot, {title: 'Total HIV Cases'});
        });
    }
});
</script>
"""

# Add the custom script right before </body>
html = html.replace("</body>", custom_script + "\n</body>")

# Save modified HTML
with open("interactive_hiv_cases_drill_down.html", "w") as f:
    f.write(html)

print("Click-to-transition behavior injected successfully!")

Interactive visualization saved. Now injecting click transition...
Click-to-transition behavior injected successfully!


## Attempt #3: breaks right down into transmission group 

In [45]:
import plotly.graph_objects as go
import pandas as pd

# Assuming df_counts is already loaded with the new columns

# Step 1: Clean column names
df_counts.columns = df_counts.columns.str.replace('\n', ' ').str.replace(' +', ' ', regex=True).str.strip()

# Step 2: Subset to the first row
first_row = df_counts.iloc[0]

# Step 3: Extract values
total_cases = int(first_row['New Diagnoses National Cases'])

# Extract transmission mode values
msm_cases = int(first_row['New Diagnoses MSM Cases'])
msm_idu_cases = int(first_row['New Diagnoses MSM/IDU Cases'])
idu_cases = int(first_row['New Diagnoses IDU Cases'])
hetero_cases = int(first_row['New Diagnoses Heterosexual Contact Cases'])
other_cases = int(first_row['New Diagnoses Other Transmission Category Cases'])

# Calculate percentages
msm_percent = (msm_cases / total_cases) * 100
msm_idu_percent = (msm_idu_cases / total_cases) * 100
idu_percent = (idu_cases / total_cases) * 100
hetero_percent = (hetero_cases / total_cases) * 100
other_percent = (other_cases / total_cases) * 100

# Step 4: Build dataframes for all levels
# Level 1: Total
data_total = pd.DataFrame({
    'category': ['Total'],
    'count': [total_cases],
    'level': ['total']
})

# Level 2: Transmission Categories
# Sort by count from highest to lowest
trans_categories = ['MSM', 'Heterosexual Contact', 'IDU', 'MSM/IDU', 'Other']
trans_counts = [msm_cases, hetero_cases, idu_cases, msm_idu_cases, other_cases]
trans_percents = [msm_percent, hetero_percent, idu_percent, msm_idu_percent, other_percent]

# Create a DataFrame and sort by count
data_trans = pd.DataFrame({
    'category': trans_categories,
    'count': trans_counts,
    'percent': trans_percents,
    'level': ['trans'] * len(trans_categories)
})
data_trans = data_trans.sort_values('count', ascending=False).reset_index(drop=True)

# Step 5: Build the figure with two traces (levels)
fig = go.Figure()

# Add first trace (Total)
fig.add_trace(go.Bar(
    x=data_total['category'],
    y=data_total['count'],
    text=[f'Total: {total_cases:,} (100%)'],
    textposition='outside',
    textfont=dict(color='white', size=14),
    marker_color='#FF3333',  # Bright red for total
    name='Total',
    hovertemplate='<b>%{x}</b><br>Count: %{y:,}<extra></extra>'
))

# Create gradient of salmon colors from darkest to lightest based on value
salmon_colors = [
    '#FF8C8C',  # Darkest salmon for highest value
    '#FFA5A5',
    '#FFBFBF',
    '#FFD8D8',
    '#FFF0F0'   # Lightest salmon for lowest value
]

# Trim or extend color list to match actual number of categories
color_palette = []
for i in range(len(data_trans)):
    color_idx = min(i, len(salmon_colors) - 1)
    color_palette.append(salmon_colors[color_idx])

# Add second trace (Transmission Categories)
fig.add_trace(go.Bar(
    x=data_trans['category'],
    y=data_trans['count'],
    text=[f'{cat}: {count:,} ({percent:.1f}%)' for cat, count, percent in 
          zip(data_trans['category'], data_trans['count'], data_trans['percent'])],
    textposition='outside',
    textfont=dict(color='white', size=12),
    marker_color=color_palette,
    visible=False,
    name='By Transmission Mode',
    hovertemplate='<b>%{x}</b><br>Count: %{y:,}<br>Percent: %{text}<extra></extra>'
))

# Add legend annotation for the transmission view
legend_text = "<b>MSM</b>: Men who have Sex with Men<br><b>IDU</b>: Injection Drug Use"
legend_annotation = dict(
    x=0.99,
    y=0.99,
    xref="paper",
    yref="paper",
    text=legend_text,
    showarrow=False,
    font=dict(color='white', size=12),
    align="right",
    bgcolor="black",
    bordercolor="white",
    borderwidth=2,
    borderpad=6,
    opacity=0.8
)

# Layout with styling and NO horizontal grid lines
fig.update_layout(
    title={
        'text': 'HIV Cases by Transmission Mode',
        'font': {'color': 'white', 'size': 24},
        'x': 0.5
    },
    xaxis_title='',
    yaxis_title={
        'text': 'Number of Cases',
        'font': {'color': 'white', 'size': 16}
    },
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    updatemenus=[
        {
            'buttons': [
                {
                    'label': 'Total',
                    'method': 'update',
                    'args': [
                        {'visible': [True, False]},
                        {
                            'title': {'text': 'Total HIV Cases', 'font': {'color': 'white', 'size': 24}},
                            'annotations': []  # No legend in total view
                        }
                    ]
                },
                {
                    'label': 'By Transmission',
                    'method': 'update',
                    'args': [
                        {'visible': [False, True]},
                        {
                            'title': {'text': 'HIV Cases by Transmission Mode', 'font': {'color': 'white', 'size': 24}},
                            'annotations': [legend_annotation]  # Add legend in transmission view
                        }
                    ]
                }
            ],
            'direction': 'down',
            'pad': {'r': 10, 't': 10},
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top',
            'font': {'color': 'white'},
            'bgcolor': '#333333',
            'bordercolor': '#555555'
        }
    ]
)

# Remove horizontal grid lines
fig.update_yaxes(
    showgrid=False,
    zeroline=False
)

# Update layout to ensure text is visible
fig.update_layout(
    margin=dict(t=100, b=100, l=50, r=50),
    autosize=True,
    height=700  # Increase height to ensure text fits
)

# Save the figure first
fig.write_html("interactive_hiv_cases_transmission.html", include_plotlyjs='cdn')

print("Interactive visualization saved. Now injecting click transition...")

# Inject custom JavaScript to handle clicks
with open("interactive_hiv_cases_transmission.html", "r") as f:
    html = f.read()

# New custom script for two-level drill-down
custom_script = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var currentView = 0;
    var plot = document.getElementsByClassName('plotly-graph-div')[0];
    
    if (plot) {
        plot.on('plotly_click', function() {
            currentView = (currentView + 1) % 2;
            var update = {};
            if (currentView === 0) {
                update = {visible: [true, false]};
                Plotly.restyle(plot, update);
                Plotly.relayout(plot, {
                    title: {
                        text: 'Total HIV Cases', 
                        font: {color: 'white', size: 24}
                    },
                    annotations: []  // Remove legend annotation
                });
            } else {
                update = {visible: [false, true]};
                Plotly.restyle(plot, update);
                // Create legend annotation
                var legendAnnotation = {
                    x: 0.99,
                    y: 0.99,
                    xref: "paper",
                    yref: "paper",
                    text: "<b>MSM</b>: Men who have Sex with Men<br><b>IDU</b>: Injection Drug Use",
                    showarrow: false,
                    font: {color: 'white', size: 12},
                    align: "right",
                    bgcolor: "black",
                    bordercolor: "white",
                    borderwidth: 2,
                    borderpad: 6,
                    opacity: 0.8
                };
                
                Plotly.relayout(plot, {
                    title: {
                        text: 'HIV Cases by Transmission Mode', 
                        font: {color: 'white', size: 24}
                    },
                    annotations: [legendAnnotation]  // Add legend annotation
                });
            }
        });
    }
});
</script>
"""

# Add the custom script right before </body>
html = html.replace("</body>", custom_script + "\n</body>")

# Save modified HTML
with open("../../interactive_viz_outputs/interactive_hiv_cases_transmission.html", "w") as f:
    f.write(html)

print("Click-to-transition behavior injected successfully!")

Interactive visualization saved. Now injecting click transition...
Click-to-transition behavior injected successfully!
